In [174]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [175]:
"""
- opens isd-history.csv
- creates new "ID" field = "USAF"+"WBAN"
- Deletes records w/ both lat and lon == 0 or lat and lon == null
- Makes BEGIN and END dates into datetime objects and creates Timespan
- Deletes records that exist for less than 1 day
- Starts with 29,963 records and winnows the dataset down to 27,813 recores
- Writes to file "ID-Info.csv", which is basically "isd-history" with the records mentioned above
  removed
- Writes to file "IDs_27,813.csv", which is simply a list of the remaining IDs.

""" 


'\n- opens isd-history.csv\n- creates new "ID" field = "USAF"+"WBAN"\n- Deletes records w/ both lat and lon == 0 or lat and lon == null\n- Makes BEGIN and END dates into datetime objects and creates Timespan\n- Deletes records that exist for less than 1 day\n- Starts with 29,963 records and winnows the dataset down to 27,813 recores\n- Writes to file "ID-Info.csv", which is basically "isd-history" with the records mentioned above\n  removed\n- Writes to file "IDs_23686.csv", which is simply a list of the remaining IDs.\n\n'

In [176]:
#From the ISD docs:

#Positive latitudes are north of the equator, negative latitudes are south of the equator. 
#Positive longitudes are east of Prime Meridian, negative longitudes are west of the Prime Meridian. 
#Latitude and longitude are usually expressed in that sequence, latitude before longitude.

In [177]:
os.getcwd()

'/Users/andymetcalf/Documents/research/wind/globalWindTrends/scripts/dataCleaning'

In [178]:
os.listdir()

['removeNoLatLon.ipynb', '.DS_Store', 'ID_Lat-Lon.ipynb', '.ipynb_checkpoints']

In [179]:
dat = pd.read_csv("../../data_light/isd-history.csv",dtype={'USAF':str, 'WBAN':str, 'STATION NAME':str,
                                                           'BEGIN':str, 'END':str})

In [180]:
len(dat) #orginal dataset length = 29,963

29963

In [181]:
dat = dat.dropna(subset=['LAT','LON']) #get only the entries that have non-null entries for lat/lon

In [182]:
len(dat) #dropping the entries that have null values results in 28,648 entries

28646

In [183]:
#we need to create a ID that is USAF-WBAN

In [184]:
dat['ID'] = ""

In [185]:
def makeID(dat):
    return str(dat["USAF"])+"-"+ str(dat["WBAN"])

In [186]:
dat["ID"] = dat.apply(makeID,axis =1)

In [187]:
dat.shape

(28646, 12)

In [188]:
dat.head(2)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,ID
2,007018,99999,WXPOD 7018,NaN,NaN,NaN,0.0,0.0,7018.0,20110309,20130730,007018-99999
4,007026,99999,WXPOD 7026,AF,NaN,NaN,0.0,0.0,7026.0,20120713,20170506,007026-99999


In [189]:
len(np.unique(dat["ID"])) #check to make sure we have unique ID

28646

In [190]:
#Now we need to make sure we get rid of the lat and longitudes that are equal to zero

In [191]:
#we need to get rid of the rows with "lat-lon" == 0.0-0.0
dat = dat[(dat.LAT !=0)&(dat.LON !=0)]


In [192]:
len(dat) #getting rid of the zero values for lat/lon results in 28,228 values

28228

In [193]:
dat.head(2)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,ID
18,008268,99999,WXPOD8278,AF,NaN,NaN,32.950,65.567,1156.7,20100519,20120323,008268-99999
40,010010,99999,JAN MAYEN(NOR-NAVY),NO,NaN,ENJA,70.933,-8.667,9.0,19310101,20170522,010010-99999


In [194]:
#I want to move the ID field to the front
cols = dat.columns.tolist()

In [195]:
cols = cols[-1:]+cols[:-1]
cols

['ID',
 'USAF',
 'WBAN',
 'STATION NAME',
 'CTRY',
 'STATE',
 'ICAO',
 'LAT',
 'LON',
 'ELEV(M)',
 'BEGIN',
 'END']

In [196]:
dat = dat[cols]
dat.head(2)

,ID,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
18,008268-99999,008268,99999,WXPOD8278,AF,NaN,NaN,32.950,65.567,1156.7,20100519,20120323
40,010010-99999,010010,99999,JAN MAYEN(NOR-NAVY),NO,NaN,ENJA,70.933,-8.667,9.0,19310101,20170522


In [197]:
#now we need to convert the BEGIN and END columns to datetime objects

In [198]:
def makeDate(dat_column):
    return datetime.strptime(dat_column, '%Y%m%d')

In [199]:
dat["BEGIN"] = dat.BEGIN.apply(makeDate)

In [200]:
dat["END"] = dat.END.apply(makeDate)

In [201]:
dat.head(2)

,ID,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
18,008268-99999,008268,99999,WXPOD8278,AF,NaN,NaN,32.950,65.567,1156.7,2010-05-19,2012-03-23
40,010010-99999,010010,99999,JAN MAYEN(NOR-NAVY),NO,NaN,ENJA,70.933,-8.667,9.0,1931-01-01,2017-05-22


In [202]:
#function to make Timespan
def makeTimespan (dat):
    return (dat.END - dat.BEGIN).days

In [203]:
dat["Timespan"] = ""

In [204]:
dat["Timespan"] = dat.apply(makeTimespan,axis = 1)

In [205]:
dat.head(3)

,ID,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END,Timespan
18,008268-99999,008268,99999,WXPOD8278,AF,NaN,NaN,32.950,65.567,1156.7,2010-05-19,2012-03-23,674
40,010010-99999,010010,99999,JAN MAYEN(NOR-NAVY),NO,NaN,ENJA,70.933,-8.667,9.0,1931-01-01,2017-05-22,31553
42,010014-99999,010014,99999,SORSTOKKEN,NO,NaN,ENSO,59.792,5.341,48.8,1986-11-20,2017-05-22,11141


In [206]:
#get rid of the entries with timespan = 0
dat = dat[dat.Timespan>0]

In [207]:
len(dat)

27813

In [208]:
dat.to_csv("../../data_light/ID_Info.csv",index=False)

In [209]:
#we need a list of the unique id's that came out of this first part of data cleaning

#first of all, how many stations are left?
len(dat)

27813

In [210]:
#okay, now let's get a list of the id's
ids = dat['ID'].values
type(ids), len(ids)

(numpy.ndarray, 27813)

In [211]:
#now let's write it to file
np.savetxt('../../data_light/IDs_27813.csv',ids,fmt='%s',delimiter=',')